# EBM: event-based model of disease progression

## Objectives:

This notebook demonstrates how to fit an event-based model of disease progression using publicly available software and simulated data:
- Load a CSV table of disease features (biomarkers), possibly from a longitudinal study
- Prepare the input data: select a subset of features; perform some basic statistical checks; etc.
- Fit the model
- Perform cross-validation

## The set-up

This notebook may be run in the cloud, or via a specially prepared conda environment (_FIXME: implement this_)

In [1]:
# Set up
import pandas as pd
import numpy as np

from pyebm import debm


## Part I: ...


__💬 QUESTION 💬 Store the result of 1 + 1 in the variable A__

__ℹ️ INFORMATION ℹ️ Have two cell, one for the question, the other with the answer__


In [3]:
# Your answer here

In [2]:
# –––––––––––––––– #
# –––– Answer –––– #
# –––––––––––––––– #

A = 1 + 1

__⚡ REMARK ⚡ Congrats__